In [23]:
%matplotlib inline
print(__doc__)
from PIL import Image
import glob
import re
import os
import csv

Automatically created module for IPython interactive environment


In [24]:
class Flip:
    """flip the image to TRIPLE the images"""
    X_rotate_path = "./data/rotate_images/"    # rotated images path
    y_rotate_path = "./data/train_rotate_solution.csv"    # rotated solution path
    
    X_flip_path = "./data/flip_images/"   # flip pic path
    y_flip_path = "./data/train_flip_solution.csv"    # flip solution path
    
    
    def flip(self, X_flip_path, y_flip_path, X_rotate_path, y_rotate_path):
        self.flip_image(X_flip_path, X_rotate_path)
        self.add_flip_solution(y_flip_path, y_rotate_path)
        
    def flip_image(self, X_flip_path, X_rotate_path):
        """flip the images and store"""
        if(not os.path.exists(X_flip_path)):
            os.makedirs(X_flip_path)
        pic_paths = self.get_pic_paths(X_rotate_path+"*.tif")
        flip_folder = X_flip_path.split('/')[2]
        for path in pic_paths:
            img = Image.open(path)
            path = path.split('/')
            path[2] = flip_folder
            path = '/'.join(path)
            
            number = int(self.get_number(path))
            
            temp_flip_path = self.change_number(path, number*2-1)
            img.save(temp_flip_path)
            
            temp_flip_path = self.change_number(path, number*2)
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
            img.save(temp_flip_path)
        
    def add_flip_solution(self, y_flip_path, y_rotate_path):
        """add flipped pics responding solution"""
        csvfile = open(y_rotate_path)
        reader = csv.reader(csvfile)
        headers = reader.next()
        
        flip_list  = []
        
        for row in reader:
            index = int(row[0])
            for i in range(index*2 - 1, index*2 + 1): # add extra solutions
                flip_row = [i, row[1]]
                flip_list.append(flip_row)
                
        with open(y_flip_path, 'w+') as flip_csvfile:
            writer = csv.DictWriter(flip_csvfile, fieldnames=headers)
            writer.writeheader()
            for row in flip_list:
                temp = dict(zip(headers, row))
                writer.writerow(temp)
        
    def get_pic_paths(self, X_path):
        """get pic paths of the target path"""
        pic_paths = []  # pic path
        for pic in glob.glob(X_path):
            pic_paths.append(pic)
            
        pic_paths.sort(self.sort_path)
        return pic_paths
    
    def get_number(self, path):
        """get number from a string"""
        pattern = re.compile(r"\d+")
        num = int(pattern.search(path).group())
        return num
    
    def change_number(self, path, number):
        """change the number inside the string"""
        return re.sub(r'\d+', str(number), path)
    
    def sort_path(self, path1, path2):
        """sort the pic paths, make them in increase queue"""
        num1 = self.get_number(path1)
        num2 = self.get_number(path2)
        return cmp(num1, num2)

In [25]:
fp = Flip()

# flip
fp.flip( fp.X_flip_path, fp.y_flip_path, fp.X_rotate_path, fp.y_rotate_path)